<img src="images/banner.png" style="width: 100%;">

# Working with Databases II Notebook 2

References:

[1] McKinney, Wes. *Python for data analysis.* " O'Reilly Media, Inc.", 2022.

[2] Teate, Renee MP. *SQL for Data Scientists: A Beginner's Guide for Building Datasets for Analysis.* John Wiley & Sons, 2021.

[3] Forta, Ben. *Sams Teach Yourself SQL in 10 Minutes a Day, 5th Edition*. O'Reilly Media, Inc., 2020

[4] Python sqlite3 documentation - https://docs.python.org/3/library/sqlite3.html

[5] Revised and grammar checked using ChatGPT - https://chatgpt.com/

Prepared by: Leodegario Lorenzo II

In the previous notebooks, we learned how to extensively use the `SELECT` statement, which undoubtedly the most frequently used SQL statement. However, there will come a time wherein you would want to do operations other than retrieving data from a database. Here, we demonstrate some of those operations, which are - inserting data, updating and deleting tables, and creating and manipulating tables.

We'll still use the farmer's market database as our database for examples which we will extend by performing the aforementioned database operations.

In [1]:
import sqlite3

import pandas as pd
import sqlalchemy as sqla

In [2]:
# !rm data/farmers_market_extended.db
# !sqlite3 data/farmers_market_extended.db < sql/farmers_market_db.sql

# Use a more portable approach for the above functions
os.remove('data/farmers_market_extended.db')
with open('sql/farmers_market_db.sql') as f:
    sql_script = f.read()

with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.executescript(sql_script)

In [3]:
db = sqla.create_engine('sqlite:///data/farmers_market_extended.db')

## 1 Inserting Data

To insert rows to a database table, we use the `INSERT INTO` statement. This statement can be used in several ways:

- Inserting a single complete row
- Inserting a single partial row
- Inserting the results of a query

Additionally, we will show how we can ***insert multiple rows programatically*** in Python using `sqlite3`.

`INSERT INTO` follows the syntax:

```sql
INSERT INTO <table name>
VALUES <row to insert>
```

Here, the `VALUES` can be replaced with an SQL query which we will show in a later subsection.

### Inserting Complete Rows

Let's look at a scenario wherein we would like to insert a new customer in the farmer's market database. As a quick inspection, we show the latest 10 entries in the `customer` table.

In [4]:
query = """
        SELECT *
        FROM customer
        ORDER BY customer_id DESC
        LIMIT 10
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,26,Tracie,Goehring,22821
1,25,Bonnie,Hassan,22801
2,24,Dawn,Nale,22801
3,23,Alvin,Laurie,22801
4,22,George,Rai,22801
5,21,Duane,Sipp,22801
6,20,Valerie,Loftis,22802
7,19,Iva,Kienzler,22802
8,18,Jeri,Mitchell,22802
9,17,Carlos,Diaz,22802


Let's say we want to insert a new customer in the farmer's market with the following details:

```
customer_id: 27
customer_first_name: Leode
customer_last_name: Gario
customer_zip: 39037
```

The simplest SQL query to accomplish this is:

```sql
INSERT INTO customer
VALUES (
    27,
    'Leode',
    'Gario',
    '39037'
)
```

The above query will work, however, in order to prevent ambiguity, it is best practice to specfy the column names explicitly after the table name.

In [5]:
query = """
        INSERT INTO customer (
            customer_id,
            customer_first_name,
            customer_last_name,
            customer_zip
        )
        VALUES (
            27,
            'Leode',
            'Gario',
            '39037'
        );
        """

For us to be able to execute this SQL script in Python, we use `sqlite3` to connect to the database and perform the query. Once again, the first step is establishing a connection:

In [6]:
conn = sqlite3.connect('data/farmers_market_extended.db')

Then we can use the `execute` command of the SQL connection object.

In [7]:
cursor = conn.execute(query)

To save the changes we've made with the database, we commit these changes to the connection, then subsequently close the connection.

In [8]:
conn.commit()

In [9]:
conn.close()

Now, the new data will appear in our query earlier.

In [10]:
query = """
        SELECT *
        FROM customer
        ORDER BY customer_id DESC
        LIMIT 10
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,27,Leode,Gario,39037
1,26,Tracie,Goehring,22821
2,25,Bonnie,Hassan,22801
3,24,Dawn,Nale,22801
4,23,Alvin,Laurie,22801
5,22,George,Rai,22801
6,21,Duane,Sipp,22801
7,20,Valerie,Loftis,22802
8,19,Iva,Kienzler,22802
9,18,Jeri,Mitchell,22802


### Inserting a single partial row

By specifying the column names after the `INSERT INTO` clause, it also allows you to provide values only for selected columns. Provided that one of the following conditions must exist:

- The column is defined as allowing `NULL` values (no value at all).
- A default value is specified in the table definition. This means, the default value will be used if no value is specified.

As an example, let's insert an incomplete row with the following information:

```
customer_first_name: GinoB
customer_zip: 80416
```

In [11]:
with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.execute("""
                 INSERT INTO
                     customer(
                         customer_first_name,
                         customer_zip
                     )
                 VALUES (?, ?)
                 """, ('GinoB', '80416'))

<div class="alert alert-warning" role="alert">

  <b>IMPORTANT ‼️ ALWAYS use placeholders `?` when specifying variables inside an SQL query</b>

  Notice here that we use a placeholder `?` in the SQL query which will be filled according to the corresponding value in the given tuple.
  
  We do this instead of using string formatting like `f`-strings, e.g. `f"INSERT ... {value}"`, to *prevent* **malicious SQL code injection**.

</div>

We verify that the row has indeed been inserted into our database. Here, the `customer_id` was given automatically a `28` value since `customer_id` is a `PRIMARY KEY` that `AUTOINCREMENT`s. Check the schema `farmers_market_db.sql` for the exact definition of the table that allows this.

In [12]:
query = """
        SELECT *
        FROM customer
        ORDER BY customer_id DESC
        LIMIT 10
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,28,GinoB,None,80416
1,27,Leode,Gario,39037
2,26,Tracie,Goehring,22821
3,25,Bonnie,Hassan,22801
4,24,Dawn,Nale,22801
5,23,Alvin,Laurie,22801
6,22,George,Rai,22801
7,21,Duane,Sipp,22801
8,20,Valerie,Loftis,22802
9,19,Iva,Kienzler,22802


### Inserting multiple rows

We can also insert multiple rows in Python using the `executemany` method of the SQL connection object.

In [13]:
rows_to_insert = [
    ('Oguri', 'Cap'),
    ('Belno', 'Light'),
    ('Tamamo', 'Cross'),
    ('Fujimasa', 'March'),
    ('Symboli', 'Rudolf')
]

In [14]:
query = """
        INSERT INTO customer(
            customer_first_name,
            customer_last_name
        )
        VALUES (?, ?)
        """

In [15]:
with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.executemany(query, rows_to_insert)

We verify that the rows have been indeed inserted into our customer's database:

In [16]:
query = """
        SELECT *
        FROM customer
        ORDER BY customer_id DESC
        LIMIT 10
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,33,Symboli,Rudolf,None
1,32,Fujimasa,March,None
2,31,Tamamo,Cross,None
3,30,Belno,Light,None
4,29,Oguri,Cap,None
5,28,GinoB,None,80416
6,27,Leode,Gario,39037
7,26,Tracie,Goehring,22821
8,25,Bonnie,Hassan,22801
9,24,Dawn,Nale,22801


### Inserting the results of a query

Another form of `INSERT` is to use it in conjunction with the results of a `SELECT` query. This is known as the `INSERT SELECT` statement.

Let's say we have a table of `new_customers` which we want to insert to the `customer` table.

In [17]:
query = """
        SELECT *
        FROM new_customers
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,1,Michelle,My Baby,10822
1,2,Obey,Your Master,12822
2,3,God,Hannibal,12822
3,4,Gold,City,10822
4,5,Haru,Urara,10822


We can insert these rows as a result of a `SELECT` query within an `INSERT` statement.

In [18]:
query = """
        INSERT INTO customer(
            customer_first_name,
            customer_last_name,
            customer_zip
        )

        SELECT
            customer_first_name,
            customer_last_name,
            customer_zip
        FROM new_customers
        """

We execute the query onto our extended farmer's market database:

In [19]:
with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.execute(query)

Which we verify the result using the following query:

In [20]:
query = """
        SELECT *
        FROM customer
        ORDER BY customer_id DESC
        LIMIT 10
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,38,Haru,Urara,10822
1,37,Gold,City,10822
2,36,God,Hannibal,12822
3,35,Obey,Your Master,12822
4,34,Michelle,My Baby,10822
5,33,Symboli,Rudolf,None
6,32,Fujimasa,March,None
7,31,Tamamo,Cross,None
8,30,Belno,Light,None
9,29,Oguri,Cap,None


## 2 Updating and Deleting Data

### Updating Data

To update or modify data in a table, we use the `UPDATE` statement, which can be done in two ways:

- To update specific rows in a table
- To update all rows in a table

The syntax of the `UPDATE` statement is as follows,

```sql
UPDATE <table_name>
SET <column_name> = <new_value>
WHERE <filter_condition>
```

As you can see, we specify three different things in the `UPDATE` statement:

- The table to be updated
- The column names and their new values
- The filter condition that determines which rows should be updated

<div class="alert alert-warning" role="alert">

  <b>IMPORTANT ‼️ DON'T Omit the `WHERE` Clause</b>

  When using `UPDATE`, special care must be taken since it is too easy to mistakenly update every row in your table.

</div>

As an example, let's fix the data entry for one row of the `customer` table, specifically, `customer_id` = `28`.

In [21]:
query = """
        SELECT *
        FROM customer
        WHERE customer_id = 28
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,28,GinoB,None,80416


Let's add a last name for this customer:

In [22]:
query = """
        UPDATE customer
        SET customer_last_name = 'Borja'
        WHERE customer_id = 28
        """

with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.execute(query)

In [23]:
query = """
        SELECT *
        FROM customer
        WHERE customer_id = 28
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,28,GinoB,Borja,80416


That's not quite right, so we also demonstrate how to update multiple columns in a row:

In [24]:
query = """
        UPDATE customer
        SET
            customer_last_name = 'Borja',
            customer_first_name = 'Benjur Emmanuel'
        WHERE customer_id = 28
        """

with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.execute(query)

In [25]:
query = """
        SELECT *
        FROM customer
        WHERE customer_id = 28
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,28,Benjur Emmanuel,Borja,80416


You can also set a `NULL` value to a column to "delete" the value. Note that `NULL` is to save no value which is very different from saving an empty string `''`. An empty string is still a value whereas `NULL` means there is no value at all.

We also demonstrate that you can update values for multiple rows, by expanding the coverage of the filter condition. Let's say we got new information about the zip codes of the newly created customers in our database:

In [26]:
query = """
        SELECT *
        FROM customer
        WHERE customer_id BETWEEN 29 AND 33
        ORDER BY customer_id DESC
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,33,Symboli,Rudolf,None
1,32,Fujimasa,March,None
2,31,Tamamo,Cross,None
3,30,Belno,Light,None
4,29,Oguri,Cap,None


In [27]:
query = """
        UPDATE customer
        SET customer_zip = 3173
        WHERE customer_id BETWEEN 29 AND 33
        """

with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.execute(query)

We verify that the value for each row has been modified accordingly:

In [28]:
query = """
        SELECT *
        FROM customer
        WHERE customer_id BETWEEN 29 AND 33
        ORDER BY customer_id DESC
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,33,Symboli,Rudolf,3173
1,32,Fujimasa,March,3173
2,31,Tamamo,Cross,3173
3,30,Belno,Light,3173
4,29,Oguri,Cap,3173


If we want to specify different values for each row, we can use instead the `executemany` method of the SQL connection object once again.

In [29]:
query = """
        UPDATE customer
        SET customer_zip = :customer_zip
        WHERE customer_id = :customer_id
        """
values = [{'customer_id': 29, 'customer_zip': 3174},
          {'customer_id': 30, 'customer_zip': 3175},
          {'customer_id': 31, 'customer_zip': 3176},
          {'customer_id': 32, 'customer_zip': 3177},
          {'customer_id': 33, 'customer_zip': 3178}]

with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.executemany(query, values)

We verify that the values have been assigned as expected:

In [30]:
query = """
        SELECT *
        FROM customer
        WHERE customer_id BETWEEN 29 AND 33
        ORDER BY customer_id DESC
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,33,Symboli,Rudolf,3178
1,32,Fujimasa,March,3177
2,31,Tamamo,Cross,3176
3,30,Belno,Light,3175
4,29,Oguri,Cap,3174


### Deleting Data

Deleting a row (or ALL rows from a table) is a much easier operation thru the use of the `DELETE` statement. The syntax of `DELETE` statement is as follows:

```sql
DELETE FROM <table_name>
WHERE <filter_condition>
```

<div class="alert alert-warning" role="alert">

  <b>IMPORTANT ‼️ DON'T Omit the `WHERE` Clause</b>

  Same as the `UPDATE` statement, special care must be taken when using `DELETE FROM` statement since it is too easy to mistakenly delete multiple or every row in your table unintentionally.

</div>

Let's say you want to delete the row containing `Haru` `Urara`,

In [31]:
query = """
        SELECT *
        FROM customer
        WHERE customer_id = 38
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,38,Haru,Urara,10822


We use the following query:

In [32]:
query = """
        DELETE FROM customer
        WHERE customer_id = 38
        """

with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.execute(query)

Notice that when we perform the previous query, the row has now been deleted.

In [33]:
query = """
        SELECT *
        FROM customer
        WHERE customer_id = 38
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip


Let's say we want `Haru` `Urara` back,

In [34]:
query = """
        INSERT INTO customer(
            customer_first_name,
            customer_last_name,
            customer_zip
        )
        VALUES (
            'Haru',
            'Urara',
            '10822'
        )
        """

with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.execute(query)

Notice that the new customer will be given a new `customer_id` (`38` was skipped) due to the automatically incrementing feature of the `PRIMARY KEY` `customer_id` column.

In [35]:
query = """
        SELECT *
        FROM customer
        ORDER BY customer_id DESC
        LIMIT 5
        """
pd.read_sql(query, db)

,customer_id,customer_first_name,customer_last_name,customer_zip
0,39,Haru,Urara,10822
1,37,Gold,City,10822
2,36,God,Hannibal,12822
3,35,Obey,Your Master,12822
4,34,Michelle,My Baby,10822


Another safety feature that should always be utilized is the enforcing of foreign key constraints. When foreign keys are present, DBMS can use them to enforce referential integrity. Meaning, if a row in a *parent* table contains data that is being referenced as a foreign key from another table, the `DELETE` statement would throw an error and be aborted.

In [36]:
query = """
        DELETE FROM customer
        WHERE customer_id = 14
        """

with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.execute("PRAGMA foreign_keys = ON;")
    conn.execute(query)

IntegrityError: FOREIGN KEY constraint failed

Here, the delete statement has been aborted since `customer_id` `14` is being referred to in the table `customer_purchases`.

In [37]:
query = """
        SELECT *
        FROM customer_purchases
        WHERE customer_id = 14
        LIMIT 10
        """
pd.read_sql(query, db)

,product_id,vendor_id,market_date,customer_id,quantity,cost_to_customer_per_qty,transaction_time
0,1,7,2019-07-03,14,0.99,6.99,17:32:00
1,1,7,2019-07-03,14,2.18,6.99,18:23:00
2,1,7,2019-07-06,14,3.04,6.99,13:05:00
3,1,7,2019-07-24,14,0.39,6.99,17:16:00
4,1,7,2019-09-07,14,0.03,6.99,09:09:00
5,1,7,2020-07-15,14,3.88,6.99,18:51:00
6,1,7,2020-08-26,14,0.91,6.99,17:35:00
7,1,7,2020-09-02,14,2.68,6.99,16:41:00
8,1,7,2020-09-12,14,3.38,6.99,13:14:00
9,1,7,2020-09-26,14,3.91,6.99,10:32:00


## 3 Creating and Manipulating Tables

### Creating Tables

To create a table, we use the `CREATE TABLE` statement, which has the following syntax:

```sql
CREATE TABLE <table_name> (
    <column1_name> <column1_dtype> <other_options>,
    <column2_name> <column2_dtype> <other_options>,
    ...
    <table_constraints>
);
```

Here we specify the following information:

- The name of the new table specified after the keywords `CREATE TABLE`
- The name and definition of the table columns separated by commas.
- Definition of table constraints such as foreign keys and other RDBMS specific options



As an example, let's look at how the `product` table was created in our sql script (found in `sql/farmers_market_db.sql`):

```sql
CREATE TABLE product (
  product_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  product_name varchar(45) DEFAULT NULL,
  product_size varchar(45) DEFAULT NULL,
  product_category_id int(11) NOT NULL,
  product_qty_type varchar(45) DEFAULT NULL,
  FOREIGN KEY (product_category_id) REFERENCES product_category (product_category_id) ON DELETE RESTRICT
);
```

Here, we see the following details:

- The table name has been defined as `product`.
- The columns are `product_id`, `product_name`, `product_size`, `product_category_id`, and `product_qty_type`. Each data type of each column was also defined correspondingly - `INTEGER`, `VARCHAR(45)`, `VARCHAR(45)`, `INT(11)`, `VARCHAR(45)`. For all the possible datatypes that you can use to define your columns, see [Datatypes in SQLite](https://www.sqlite.org/datatype3.html) for more details.
- The option whether the column allows `NULL` values is also specified. If a column is set to a `NULL` column, it allows for null values, if we set `NOT NULL` to a column, it does not accept rows with no value.
- For rows that allow `NULL` values, the `DEFAULT` value was also set to `NULL`
- The `product_id` column was also defined as the `PRIMARY KEY` of the table, with an optional `AUTOINCREMENT` feature, which we saw in action when we insert new rows to the table.
- Finally, we also see definitions regarding constraints present in the table which are its foreign keys along with the table that it is refering to. Also the `ON DELETE RESTRICT`, restricts the deletion of rows in the parent table (`product_category_id`) with data that is being referred to or used by the child table (`product`) table.

As a worked example, let's add a new table to the farmer's market database named `promo` which will contain promotional events that may occur in the market.

Let's first use the following query to check all of the tables present in our database:

In [38]:
query = """
        SELECT *
        FROM sqlite_master
        WHERE type='table'
        """
pd.read_sql(query, db)

,type,name,tbl_name,rootpage,sql
0,table,booth,booth,2,CREATE TABLE booth (\n booth_number int(11) N...
1,table,customer,customer,4,CREATE TABLE customer (\n customer_id INTEGER...
2,table,sqlite_sequence,sqlite_sequence,5,"CREATE TABLE sqlite_sequence(name,seq)"
3,table,new_customers,new_customers,6,CREATE TABLE new_customers (\n customer_id IN...
4,table,vendor,vendor,7,CREATE TABLE vendor (\n vendor_id INTEGER NOT...
5,table,product_category,product_category,8,CREATE TABLE product_category (\n product_cat...
6,table,product,product,9,CREATE TABLE product (\n product_id INTEGER N...
7,table,vendor_inventory,vendor_inventory,10,CREATE TABLE vendor_inventory (\n market_date...
8,table,customer_purchases,customer_purchases,23,CREATE TABLE customer_purchases (\n product_i...
9,table,market_date_info,market_date_info,106,CREATE TABLE market_date_info (\n market_date...


In [39]:
query = """
        CREATE TABLE promo (
            promo_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            promo_name VARCHAR(45) NOT NULL,
            promo_start DATETIME NOT NULL,
            promo_end DATETIME NOT NULL,
            promo_info VARCHAR(255) DEFAULT NULL
        )
        """
with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.execute(query)

We verify that indeed we have created the table.

In [41]:
query = """
        SELECT *
        FROM sqlite_master
        WHERE type='table'
        """
pd.read_sql(query, db)

,type,name,tbl_name,rootpage,sql
0,table,booth,booth,2,CREATE TABLE booth (\n booth_number int(11) N...
1,table,customer,customer,4,CREATE TABLE customer (\n customer_id INTEGER...
2,table,sqlite_sequence,sqlite_sequence,5,"CREATE TABLE sqlite_sequence(name,seq)"
3,table,new_customers,new_customers,6,CREATE TABLE new_customers (\n customer_id IN...
4,table,vendor,vendor,7,CREATE TABLE vendor (\n vendor_id INTEGER NOT...
5,table,product_category,product_category,8,CREATE TABLE product_category (\n product_cat...
6,table,product,product,9,CREATE TABLE product (\n product_id INTEGER N...
7,table,vendor_inventory,vendor_inventory,10,CREATE TABLE vendor_inventory (\n market_date...
8,table,customer_purchases,customer_purchases,23,CREATE TABLE customer_purchases (\n product_i...
9,table,market_date_info,market_date_info,106,CREATE TABLE market_date_info (\n market_date...


We can also use the SQLite specific pragma command `PRAGMA table_info` to retrieve information about the table we've created:

In [43]:
query = """
        PRAGMA table_info(promo)
        """
pd.read_sql(query, db)

,cid,name,type,notnull,dflt_value,pk
0,0,promo_id,INTEGER,1,None,1
1,1,promo_name,VARCHAR(45),1,None,0
2,2,promo_start,DATETIME,1,None,0
3,3,promo_end,DATETIME,1,None,0
4,4,promo_info,VARCHAR(255),0,NULL,0


Once the table has been created, we can then use the `INSERT INTO` statement to populate data into the table.

In [44]:
query = """
        INSERT INTO promo(
            promo_name,
            promo_start,
            promo_end,
            promo_info
        )
        VALUES (
            'CNY 2026',
            '2026-02-01 00:00',
            '2026-02-28 23:59',
            'Chinese New Year promotional event for the year 2026'
        )
        """
with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.execute(query)

We verify that the promo entry has been properly created in the table `promo`.

In [45]:
query = """
        SELECT
            promo_id, promo_name, promo_start, promo_end, promo_info
        FROM promo
        """
pd.read_sql(query, db)

,promo_id,promo_name,promo_start,promo_end,promo_info
0,1,CNY 2026,2026-02-01 00:00,2026-02-28 23:59,Chinese New Year promotional event for the yea...


Now that we have the first promo in the database, the next question, which will be relevant to the customers, is *What products are actually covered by this promotion?*

To answer this, we need to modify the `vendor_inventory` table to indicate which products are covered by the promotion.

### Updating Tables

To change or modify existing tables, we use the `ALTER TABLE` statement, which has the following syntax:

```sql
ALTER TABLE <table_name>
<operational clause>;
```

Here, we specify the following information:

- The name of the table to be altered
- The change to be made with specifications

<div class="alert alert-warning" role="alert">

  <b>IMPORTANT ‼️ Use `ALTER TABLE` Carefully</b>

  Use `ALTER TABLE` with extreme caution, and be sure you have backups (both the schema and data) before using it.
  
  Database table changes CANNOT be undone, and if you add columns you don't need, you might not be able to remove them. Similarly, if you drop a column that you do need, you might lose all the data in that column.

</div>

As an example, let's add the columns `promo_id` and `discount` onto the `vendor_inventory` table.

In [58]:
query = """
        ALTER TABLE vendor_inventory
        ADD COLUMN discount DECIMAL(16,2) DEFAULT NULL;

        ALTER TABLE vendor_inventory
        ADD COLUMN promo_id INTEGER DEFAULT NULL REFERENCES promo(promo_id) ON DELETE RESTRICT;
        """
with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.executescript(query)

We verify that the columns `promo_id` and `discount` were added onto the `vendor_inventory` table.

In [59]:
query = """
        PRAGMA table_info(vendor_inventory)
        """
pd.read_sql(query, db)

,cid,name,type,notnull,dflt_value,pk
0,0,market_date,date,1,None,3
1,1,quantity,"decimal(16,2)",0,NULL,0
2,2,vendor_id,int(11),1,None,2
3,3,product_id,int(11),1,None,1
4,4,original_price,"decimal(16,2)",0,NULL,0
5,5,promo_id,INTEGER,0,NULL,0
6,6,discount,"DECIMAL(16,2)",0,NULL,0


Furthermore, the foreign key constraints were also created as inteded:

In [60]:
query = """
        PRAGMA foreign_key_list(vendor_inventory)
        """
pd.read_sql(query, db)

,id,seq,table,from,to,on_update,on_delete,match
0,0,0,vendor,vendor_id,vendor_id,NO ACTION,NO ACTION,NONE
1,1,0,product,product_id,product_id,NO ACTION,NO ACTION,NONE
2,2,0,promo,promo_id,promo_id,NO ACTION,RESTRICT,NONE


As such, we can insert data onto the `vendor_inventory` table with the new columns. We demonstrate this by inserting the data found in the `vendor_inventory_update.csv` using `pandas` `to_sql` function.

In [83]:
vi_update = pd.read_csv('data/vendor_inventory_update.csv', parse_dates=['market_date'])
vi_update

,market_date,quantity,vendor_id,product_id,original_price,promo_id,discount
0,2026-02-17,20,1,10,1,1.0,20.0
1,2026-02-17,100,1,11,2,1.0,20.0
2,2026-02-17,50,6,17,1,NaN,NaN
3,2026-02-17,100,6,21,1,1.0,10.0
4,2026-02-17,100,6,22,1,1.0,10.0


In [85]:
vi_update.to_sql('vendor_inventory', db, if_exists='append', index=False)

5

In [90]:
query = """
        SELECT *
        FROM vendor_inventory
        ORDER BY market_date DESC
        LIMIT 10
        """
pd.read_sql(query, db, dtype={'promo_id': pd.Int64Dtype()})

,market_date,quantity,vendor_id,product_id,original_price,promo_id,discount
0,2026-02-17 00:00:00.000000,20,1,10,1.0,1,20.0
1,2026-02-17 00:00:00.000000,100,1,11,2.0,1,20.0
2,2026-02-17 00:00:00.000000,50,6,17,1.0,<NA>,NaN
3,2026-02-17 00:00:00.000000,100,6,21,1.0,1,10.0
4,2026-02-17 00:00:00.000000,100,6,22,1.0,1,10.0
5,2020-10-10,30,7,4,4.0,<NA>,NaN
6,2020-10-10,20,8,5,6.5,<NA>,NaN
7,2020-10-10,10,8,7,18.0,<NA>,NaN
8,2020-10-10,10,8,8,18.0,<NA>,NaN
9,2020-10-07,30,7,4,4.0,<NA>,NaN


### Other table manipulation operations

The other operations that you can do with the `ALTER TABLE` statement is listed in the following table:

| Clause | Operation |
| :----- | :-------- |
| `RENAME TO <new_table_name>` | Changes the name of `<table_name>` to `<new_table_name>` |
| `RENAME COLUMN <column_name> TO <new_column_name>` | Changes the `<column_name>` of table `<table_name>` to `<new_column_name>` |
| `DROP COLUMN <column_name>` | Remove the existing column `<column_name>` from a table |

Each of the clauses have their own specific restrictions such as restricting removal of a column if the column is a `PRIMARY KEY`, for a comprehensive list of all of the restriction, do check the documentation of `SQLite` for [ALTER TABLE](https://www.sqlite.org/lang_altertable.html).

### Deleting Tables

Finally, we show how easy it is to delete tables using the `DROP TABLE` statement which has the syntax:

```sql
DROP TABLE <table_name>
```

As an example, let's create a table which we will use as an example for deletion:

In [98]:
query = """
        CREATE TABLE fleeting (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            description TEXT
        );
        INSERT INTO fleeting (description)
        VALUES ('one'), ('two'),('three');
        """
with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.executescript(query)

Verify that we can see it present in all of the listed tables in our database.

In [99]:
query = """
        SELECT *
        FROM sqlite_master
        WHERE type='table'
        """
pd.read_sql(query, db)

,type,name,tbl_name,rootpage,sql
0,table,booth,booth,2,CREATE TABLE booth (\n booth_number int(11) N...
1,table,customer,customer,4,CREATE TABLE customer (\n customer_id INTEGER...
2,table,sqlite_sequence,sqlite_sequence,5,"CREATE TABLE sqlite_sequence(name,seq)"
3,table,new_customers,new_customers,6,CREATE TABLE new_customers (\n customer_id IN...
4,table,vendor,vendor,7,CREATE TABLE vendor (\n vendor_id INTEGER NOT...
5,table,product_category,product_category,8,CREATE TABLE product_category (\n product_cat...
6,table,product,product,9,CREATE TABLE product (\n product_id INTEGER N...
7,table,vendor_inventory,vendor_inventory,10,CREATE TABLE vendor_inventory (\n market_date...
8,table,customer_purchases,customer_purchases,23,CREATE TABLE customer_purchases (\n product_i...
9,table,market_date_info,market_date_info,106,CREATE TABLE market_date_info (\n market_date...


And also it has the following contents as expected from our table creation.

In [100]:
query = """
        SELECT *
        FROM fleeting
        """
pd.read_sql(query, db)

,id,description
0,1,one
1,2,two
2,3,three


To delete it, we simply use the `DROP TABLE` statement.

In [101]:
query = """
        DROP TABLE fleeting
        """
with sqlite3.connect('data/farmers_market_extended.db') as conn:
    conn.execute(query)

Verify that the table has now been deleted in our database.

In [102]:
query = """
        SELECT *
        FROM sqlite_master
        WHERE type='table'
        """
pd.read_sql(query, db)

,type,name,tbl_name,rootpage,sql
0,table,booth,booth,2,CREATE TABLE booth (\n booth_number int(11) N...
1,table,customer,customer,4,CREATE TABLE customer (\n customer_id INTEGER...
2,table,sqlite_sequence,sqlite_sequence,5,"CREATE TABLE sqlite_sequence(name,seq)"
3,table,new_customers,new_customers,6,CREATE TABLE new_customers (\n customer_id IN...
4,table,vendor,vendor,7,CREATE TABLE vendor (\n vendor_id INTEGER NOT...
5,table,product_category,product_category,8,CREATE TABLE product_category (\n product_cat...
6,table,product,product,9,CREATE TABLE product (\n product_id INTEGER N...
7,table,vendor_inventory,vendor_inventory,10,CREATE TABLE vendor_inventory (\n market_date...
8,table,customer_purchases,customer_purchases,23,CREATE TABLE customer_purchases (\n product_i...
9,table,market_date_info,market_date_info,106,CREATE TABLE market_date_info (\n market_date...


<img src="images/banner-down.png" style="width: 100%;">